# STOR515 Final Project Codebase
### Modeling Inventory Decisions for Stoney River Steakhouse

#### Project Authors: Alex Huml, Carter Hall, Thomas Bridges, Ian Wilson

In [3]:
%autosave 15
import numpy as np 
import statistics as stats # stats.NormalDist is the norm. dist. function w/ object 'cdf'
import pandas as pd # Pretty-printing, exporting dataframes to Excel

Autosaving every 15 seconds


In [14]:
class WeLoveSteak:
    """Modeling the demand for delicious steaks as a finite-horizon Markov Decision Process."""
    def __init__(self, M, N, mu, sigma, sellPrice, priceyint, steakName):
        """Initialization Function.
        
        Parameters:
        M := Limit on storage capacity.
        N := Number of decision epochs.
        S := Set of States, T := Set of Epochs
        mu, sigma: Normal Dist. params
        sp := Selling price 
        priceyint := Avg 2021 price
        steakName := Name of steak
        """
        self.M = M
        self.N = N
        self.S = set(range(self.M+1))
        self.T = set(range(1,self.N+1))
        self.mu = mu
        self.sigma = sigma
        self.sp = sellPrice
        self.priceyint = priceyint
        self.ustar, self.astar = self.optimality(steakName)
        self.storeResults(steakName)
        
    def A(self, s):
        '''Function representing action space.
        
        Parameters:
        s := State from state space S.
        '''
        return set(range(0,self.M - s + 1))
        
    def f(self, s):
        """Revenue generated from selling strip loins at $44/pound.
        
        Parameters:
        s := State indicating number of loins sold.
        """
        return self.sp * s
    
    def price(self, t):
        """Price for Strip Loin, adjusted for inflation via inflation rate.
        
        Parameters:
        t := Epoch in T
        """
        return self.priceyint * (1 + (0.0125 / (np.sqrt(t) - 0.5)))
    
    def CDF(self, s):
        """Function for discretized probability that uses the normal CDF.
        
        Parameters:
        s := State in S
        """
        return stats.NormalDist(self.mu, self.sigma).cdf(s)
    
    def pt(self, j, s, a):
        '''Transition probability function.
            
        Parameters:
        j,s := states in S
        a := action in A
        
        Returns:
        0 if j > s + a
        CDF(s + a - j) - CDF(s + a - j - 1) if 0 < j <= s + a
        1 - CDF(s + a) otherwise
        '''
        assert type(j) == type(s) == type(a) == type(3)
        
        if j > s + a:
            return 0
        elif 0 < j and j <= s + a:
            return self.CDF(s + a - j) - self.CDF(s + a - j - 1)
        return 1 - self.CDF(s + a)
    
    def rt(self, t, s, a = 0):
        
        if t == self.N:
            return (-s*self.price(t))
        elif s + a == 0: # if j = 0, as j = s + a
            return sum([self.CDF(s+a) * self.f(s+a) for k in range(s+a+1, self.M+1)])
        
        return -a*self.price(t) + sum([(self.CDF(k) - self.CDF(k - 1)) * self.f(k) for k in range(s+a+1)]) 
    
    def optimalAction(self, d):
        '''Function to determine which action corresponds to optimal policy.
        
            Parameters
            ----------
            d: dict -> dictionary of (action, optimality eqn. value) 
        '''
        v = max(d.values())
        for k in d.keys():
            if d[k] == v:
                return k

        raise ValueError("Should never get here, but.")
    
    def optimality(self, steakName):
        '''Implements algorithm to find optimal policy via backward induction.'''
        t = len(self.T) # From 1 to N + 1, this returns N 
        ustar = np.zeros([len(self.S), t])
        astar = np.zeros([len(self.S), t])
        
        # BC computation -- u_{N+1}^{*} (s) = r_{N} s for all s in S
        for s in self.S:
            ustar[s,-1] = self.rt(t,s)
        
        while t != 1:
            print(steakName + ": " + str(t))
            t -= 1
            for s in self.S:
                l = dict()
                for a in self.A(s):
                    temp = self.rt(t,s,a) + sum([self.pt(j,s,a) * ustar[j,t] for j in self.S])
                    l[a] = l.get(a, 0) + temp
                ustar[s,t-1] = round(max(l.values()), 4) # Rounding this because of PDF output -- doesn't change answer
                astar[s,t-1]= self.optimalAction(l)
        
        ustar = self.getOptimalityTable(ustar)
        astar = self.getOptimalPolicy(astar)
        return ustar, astar

    def getOptimalPolicy(self, astar):
        '''Pretty-prints the optimal policy dataframe.'''
        return pd.DataFrame(astar, columns = ['Week {}'.format(t) for t in range(1,self.N+1)], 
                                   index   = ['State {}'.format(s) for s in range(0,len(self.S))])
    
    def getOptimalityTable(self, ustar):
        '''Pretty-prints the total expected reward dataframe.'''
        return pd.DataFrame(ustar, columns = ['Week {}'.format(t) for t in range(1,self.N+1)], 
                                   index   = ['State {}'.format(s) for s in range(0,len(self.S))])
    
    
    def storeResults(self, steakName):
        with pd.ExcelWriter("./MDPResults" + steakName + ".xlsx") as writer:
            self.astar.to_excel(writer, sheet_name="OptPolicy", index=True)
            self.ustar.to_excel(writer, sheet_name="ExpReward", index=True)
        return

In [15]:
striploin = WeLoveSteak(168, 32, 112, 43, 44, 8.96, "striploin")

striploin: 32
striploin: 31
striploin: 30
striploin: 29
striploin: 28
striploin: 27
striploin: 26
striploin: 25
striploin: 24
striploin: 23
striploin: 22
striploin: 21
striploin: 20
striploin: 19
striploin: 18
striploin: 17
striploin: 16
striploin: 15
striploin: 14
striploin: 13
striploin: 12
striploin: 11
striploin: 10
striploin: 9
striploin: 8
striploin: 7
striploin: 6
striploin: 5
striploin: 4
striploin: 3
striploin: 2


In [16]:
cowboy = WeLoveSteak(171, 32, 54, 26, 36, 11.32, "cowboy")

cowboy: 32
cowboy: 31
cowboy: 30
cowboy: 29
cowboy: 28
cowboy: 27
cowboy: 26
cowboy: 25
cowboy: 24
cowboy: 23
cowboy: 22
cowboy: 21
cowboy: 20
cowboy: 19
cowboy: 18
cowboy: 17
cowboy: 16
cowboy: 15
cowboy: 14
cowboy: 13
cowboy: 12
cowboy: 11
cowboy: 10
cowboy: 9
cowboy: 8
cowboy: 7
cowboy: 6
cowboy: 5
cowboy: 4
cowboy: 3
cowboy: 2


In [ ]:
ribeye = WeLoveSteak(360, 32, 119, 54, 48, 12.68, "ribeye")

ribeye: 32
ribeye: 31
ribeye: 30
ribeye: 29
ribeye: 28
ribeye: 27
ribeye: 26
ribeye: 25
ribeye: 24
ribeye: 23
ribeye: 22
ribeye: 21
ribeye: 20
ribeye: 19
ribeye: 18
ribeye: 17
ribeye: 16
ribeye: 15


In [ ]:
tenderloin = WeLoveSteak(588, 32, 188, 89, 75, 14.98, "tenderloin")